### Pivot OHLCV Data for Time-Series Analysis

This notebook transforms the cleaned, multi-indexed OHLCV data from a "long" format into a "wide" format. This new format has dates as the index and tickers as columns, which is required by many backtesting and time-series analysis libraries.

**Workflow:**

1.  **Load Data:** Reads the cleaned, multi-indexed OHLCV data.
2.  **Transform:** Selects a specific column (e.g., `Adj Close`) and pivots the DataFrame.
3.  **Save:** Saves the new wide-format DataFrame to a Parquet file.
4.  **Verify:** Reads the saved file back to confirm the process was successful.

### Setup and Configuration

**This is the only cell you need to modify.** Specify the source/destination files and the data column you wish to pivot.

In [1]:
import sys
from pathlib import Path
import pandas as pd

# --- Project Path Setup ---
# Autodetect the project's root directory.
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent 
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- File and Column Configuration ---
SOURCE_FILENAME = 'df_OHLCV_clean_stocks_etfs.parquet'
DEST_FILENAME = 'df_adj_close.parquet'
COLUMN_TO_PIVOT = 'Adj Close'  # The data column to extract and pivot

# --- Construct Full Paths ---
SOURCE_PATH = DATA_DIR / SOURCE_FILENAME
DEST_PATH = DATA_DIR / DEST_FILENAME

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Project Root Directory: {ROOT_DIR}")
print(f"Source file: {SOURCE_PATH}")
print(f"Pivoting column: '{COLUMN_TO_PIVOT}'")
print(f"Destination file: {DEST_PATH}")

Project Root Directory: c:\Users\ping\Files_win10\python\py311\stocks
Source file: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet
Pivoting column: 'Adj Close'
Destination file: c:\Users\ping\Files_win10\python\py311\stocks\data\df_adj_close.parquet


### Step 1: Load Source OHLCV Data

Load the cleaned, multi-indexed data that will be transformed.

In [2]:
print(f"--- Step 1: Loading source data from {SOURCE_PATH.name} ---")

try:
    df_ohlcv = pd.read_parquet(SOURCE_PATH)
    print("Successfully loaded source data.")
    df_ohlcv.info()
    display(df_ohlcv.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}. Halting execution.")
    df_ohlcv = None
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df_ohlcv = None

--- Step 1: Loading source data from df_OHLCV_clean_stocks_etfs.parquet ---
Successfully loaded source data.
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 387000 entries, ('A', Timestamp('2024-12-27 00:00:00')) to ('ZWS', Timestamp('2025-12-26 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Open   387000 non-null  float64
 1   Adj High   387000 non-null  float64
 2   Adj Low    387000 non-null  float64
 3   Adj Close  387000 non-null  float64
 4   Volume     387000 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 16.3+ MB


,,Adj Open,Adj High,Adj Low,Adj Close,Volume
Ticker,Date,,,,,
A,2024-12-27,133.5370,135.1240,133.5070,134.4490,636924
AA,2024-12-27,37.4905,37.6484,36.9079,37.2041,1949015
AAL,2024-12-27,17.1900,17.3800,17.0100,17.3500,29500500


### Step 2: Extract and Pivot Data

This step performs the core transformation: it selects the specified data column and pivots the DataFrame from a long to a wide format.

In [ ]:
def pivot_timeseries_data(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Selects a column from a multi-indexed DataFrame and pivots it.

    Args:
        df (pd.DataFrame): Input DataFrame with a ('Ticker', 'Date') MultiIndex.
        column_name (str): The name of the column to pivot.

    Returns:
        pd.DataFrame: A wide-format DataFrame with dates as the index and
                      tickers as columns.
    """
    # 1. Select the target column (returns a Series)
    series = df[column_name]

    # 2. Unstack the 'Ticker' level (level=0) to become columns
    pivoted_df = series.unstack(level=0)

    # 3. Ensure both index (Date) and columns (Ticker) are sorted
    pivoted_df = pivoted_df.sort_index(axis=0).sort_index(axis=1)

    return pivoted_df


# --- Execute Step 2 ---
if df_ohlcv is not None:
    print(f"\n--- Step 2: Pivoting '{COLUMN_TO_PIVOT}' column ---")
    df_pivoted = pivot_timeseries_data(df_ohlcv, COLUMN_TO_PIVOT)

    print("Transformation complete. New DataFrame info:")
    df_pivoted.info()

    print("\nFirst 5 rows of pivoted data:")
    display(df_pivoted.head())
else:
    print("\nSkipping transformation step because source data failed to load.")
    df_pivoted = None


--- Step 2: Pivoting 'Adj Close' column ---
Transformation complete. New DataFrame info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 250 entries, 2024-12-27 to 2025-12-26
Columns: 1548 entries, A to ZWS
dtypes: float64(1548)
memory usage: 3.0 MB

First 5 rows of pivoted data:


Ticker            A       AA    AAL     AAON     AAPL     ABBV     ABEV     ABNB      ABT   ACGL      ACI      ACM      ACN     ACWI     ACWX    ADBE      ADC      ADI      ADM      ADP    ADSK      ADT      AEE      AEG     AEIS      AEM      AEP      AER      AES      AFG      AFL   AFRM       AG     AGCO      AGG      AGI     AGNC      AHR      AIG      AIQ     AIRR      AIT      AIZ      AJG   AKAM       AL    ALAB      ALB      ALC    ALGN      ALL     ALLE     ALLY    ALNY     ALSN      ALV       AM     AMAT     AMCR     AMD      AME      AMG     AMGN      AMH     AMKR     AMLP      AMP      AMT   AMTM      AMX    AMZN      AN    ANET      AON      AOS      APA      APD      APG      APH  APLD      APO     APP    APPF   APTV     AR     ARCC      ARE     ARES    ARGX   ARKK     ARM     ARMK   ARWR     AS     ASML    ASND      ASR   ASTS      ASX    ATI      ATO      ATR       AU   AUR    AVAV      AVB     AVDE     AVDV     AVEM     AVGO     AVLV   AVTR     AVUS     AVUV      AVY      AWI      AWK     AXIA    AXON      AXP      AXS   AXSM   AXTA      AYI      AZN      AZO        B      BA     BABA      BAC      BAH      BAI     BALL      BAM      BAP      BAX     BBAX     BBCA      BBD     BBEU     BBIN   BBIO     BBJP     BBUS     BBVA      BBY      BCE      BCH      BCS      BDX     BE     BEKE      BEN      BEP     BEPC     BF-A     BF-B       BG      BHP   BIDU    BIIB      BIL   BILI     BINC     BIO      BIP   BIRK      BIV     BJ       BK     BKLC     BKLN     BKNG      BKR     BLD    BLDR       BLK      BLV      BMO   BMRN      BMY       BN      BND     BNDX      BNS      BNT    BNTX     BOKF     BOND    BOXX       BP     BPOP       BR     BRK-A   BRK-B     BRKR       BRO   BROS      BRX     BSAC     BSBR     BSCQ     BSCR      BSV    BSX      BSY    BTC      BTI      BUD   BUFR    BURL      BVN      BWA     BWXT       BX      BXP      BYD       BZ        C    CACI     CADE    CAE      CAG      CAH     CARR   CART     CASY      CAT       CB     CBOE    CBRE     CBSH     CCEP      CCI      CCJ      CCK    CCL   CDE    CDNS      CDP      CDW    CEF      CEG   CELH       CF      CFG   CFLT      CFR       CG     CGBL     CGCP     CGDV     CGGO     CGGR     CGMU     CGUS     CGXU      CHD     CHDN    CHKP     CHRW      CHT    CHTR   CHWY       CI      CIB     CIBR   CIEN  CIFR     CIGI     CINF       CL   CLF     CLH    CLS      CLX       CM      CMA      CMC    CMCSA      CME    CMG      CMI      CMS      CNA    CNC      CNH       CNI    CNM  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

### Step 3: Save Pivoted Data

Save the newly created wide-format DataFrame to a Parquet file for future use.

In [ ]:
if df_pivoted is not None:
    print(f"\n--- Step 3: Saving pivoted data ---")
    try:
        # Ensure the destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)

        df_pivoted.to_parquet(DEST_PATH, index=True)
        print(f"Successfully saved pivoted data to: {DEST_PATH}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")
else:
    print("\nSkipping save step because the pivoted DataFrame does not exist.")


--- Step 3: Saving pivoted data ---
Successfully saved pivoted data to: c:\Users\ping\Files_win10\python\py311\stocks\data\df_adj_close.parquet


### Step 4: Verify Saved File

To ensure the pipeline ran correctly, load the saved file and display its contents.

In [ ]:
print(f"\n--- Step 4: Verifying saved file ---")

try:
    if DEST_PATH.exists():
        verified_df = pd.read_parquet(DEST_PATH)
        print("Verification successful! File read back into memory.")
        print("Last 5 rows of verified data:")
        display(verified_df.tail())
    else:
        print(
            f"ERROR: Output file not found at {DEST_PATH}. The save step may have failed."
        )
except Exception as e:
    print(f"An error occurred during verification: {e}")


--- Step 4: Verifying saved file ---
Verification successful! File read back into memory.
Last 5 rows of verified data:


Ticker           A     AA    AAL   AAON    AAPL    ABBV  ABEV    ABNB     ABT   ACGL    ACI    ACM     ACN    ACWI   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEIS     AEM     AEP     AER    AES     AFG     AFL   AFRM     AG    AGCO     AGG    AGI   AGNC    AHR    AIG    AIQ    AIRR     AIT     AIZ     AJG   AKAM     AL    ALAB     ALB    ALC    ALGN     ALL    ALLE   ALLY    ALNY    ALSN     ALV     AM    AMAT  AMCR     AMD     AME     AMG    AMGN    AMH   AMKR   AMLP     AMP     AMT   AMTM    AMX    AMZN      AN    ANET     AON    AOS    APA     APD    APG     APH   APLD     APO     APP    APPF   APTV     AR   ARCC    ARE    ARES    ARGX   ARKK     ARM   ARMK   ARWR     AS     ASML    ASND     ASR   ASTS    ASX     ATI     ATO     ATR     AU   AUR    AVAV     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR    AVUS    AVUV     AVY     AWI     AWK   AXIA    AXON     AXP     AXS    AXSM   AXTA     AYI    AZN      AZO      B      BA    BABA    BAC    BAH    BAI   BALL    BAM     BAP    BAX     BBAX    BBCA   BBD    BBEU     BBIN   BBIO     BBJP     BBUS   BBVA    BBY    BCE    BCH    BCS     BDX     BE   BEKE    BEN    BEP   BEPC   BF-A   BF-B     BG    BHP    BIDU    BIIB    BIL   BILI   BINC     BIO    BIP   BIRK    BIV     BJ      BK    BKLC    BKLN     BKNG    BKR     BLD    BLDR      BLK    BLV     BMO   BMRN    BMY     BN    BND   BNDX    BNS    BNT   BNTX    BOKF   BOND    BOXX     BP    BPOP      BR     BRK-A   BRK-B   BRKR    BRO   BROS    BRX   BSAC  BSBR    BSCQ    BSCR    BSV    BSX    BSY    BTC    BTI    BUD    BUFR    BURL    BVN    BWA    BWXT      BX    BXP    BYD     BZ       C    CACI   CADE    CAE    CAG     CAH   CARR   CART    CASY     CAT      CB    CBOE    CBRE   CBSH   CCEP    CCI    CCJ     CCK    CCL    CDE    CDNS    CDP     CDW    CEF     CEG   CELH     CF    CFG   CFLT     CFR     CG     CGBL     CGCP     CGDV     CGGO     CGGR     CGMU     CGUS     CGXU    CHD    CHDN    CHKP    CHRW    CHT    CHTR   CHWY      CI    CIB   CIBR    CIEN   CIFR    CIGI    CINF     CL    CLF     CLH     CLS    CLX     CM    CMA    CMC  CMCSA     CME    CMG     CMI    CMS    CNA    CNC   CNH    CNI    CNM    CNP    CNQ     COF    COHR    COIN    COKE   COLB    COO    COP   COPX     COR    COST   COWZ     CP    CPAY    CPB   CPNG   CPRT     CPT    CQP      CR   CRBG    CRDO     CRH    CRK     CRL     CRM     CRS    CRWD   CSCO   CSGP     CSL    CSX    CTAS   CTRA   CTRE   CTSH   CTVA   CUBE    CVE    CVNA    CVS  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             